In [56]:
import camelot
import pandas as pd
import numpy as np
import pdfminer as pm
import math
from datetime import datetime  
from datetime import timedelta  
import warnings
import os

In [7]:
print(len(p1t))

0


In [75]:
## Read in all table regions
def convertPDFToTable(filepath):
    table_cols = [['35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550',
                  '35,52,97,136,210,256,308,360,390,422,456,469,490,513,550'], 
                  ['44,']]
    curColSetting = 0
    p1t = camelot.read_pdf(filepath, 
                         strip_text=' .\n', flavor='stream', pages="1-end",
                         #table_regions=['0,700,600,0'],
                          columns=table_cols[curColSetting], 
                           edge_tol=75)
    pdf = []
    for t in p1t:
        pdf.append(t.df)
    return pdf

In [68]:
def processTables(tables, committee, station):
    progColName=False
    ## Create the ad day/times df.
    ads = pd.DataFrame(columns=['StartDate', 'EndDate', 'Weekdays', 'Spots/Week', 'Rate', 'Rating', 'index'])
    ## Create the shows DF.
    shows = pd.DataFrame(columns=['index', 'Title'])
    secondlines = pd.DataFrame(columns=['index', 'Title'])
    ## Create final adtimes DF.
    adtimes = []
    ## Add second line of inventory code
    def mergeSLines(row):
        if row.name+1 < len(shows):
            nextRow = shows.iloc[row.name+1]
            sline = secondlines[(row['index'] < secondlines['index']) & (nextRow['index']>secondlines['index'])]
        else: 
            sline = secondlines[(row['index'] < secondlines['index'])]
        if(len(sline)>0):
            if(row['Title'] != sline['Title'].values[0]):
                row['Title'] = row['Title'] + sline['Title'].values[0]
        else:
            print('Second line merge error at ad: ', row.name)
        return row
    
    def getSecondLines(df):
        returnVal = pd.DataFrame(columns=['index', 'Title'])
        #Get show lines
        progs = df[(df['Amount'] != '') & (df['Ch'] != '')]
        progs = progs.iloc[1:] # Remove residual header row.
        slines = list(map(lambda x: x+1, progs.index.values))
        if not (slines[-1] < len(df)): ## In case the second show line is on the next page.
            print('Program second line on next page.')
            slines = slines[:-1]
        slines = df.iloc[slines]
        slines = slines.replace('', np.nan, regex=True)
        slines = slines[slines.isnull().sum(axis=1) == len(slines.columns)-1]
        for index, row in slines.iterrows():
            nulls = pd.isnull(row)
            nulls = nulls[nulls!=True]
            returnVal = returnVal.append({'index': index, 'Title': row[nulls.index[0]]}, ignore_index=True)
        returnVal['index'] = returnVal['index'] + lastIndex
        return returnVal
    def addNameIfDiff(p):
        for val in nlines.iloc[p.name]:
            if(p[progColName] != val) & (val.isna() != True):
                p[progColName] = p[progColName] + val
        return p
    ## Merge program names into ad days.
    def mergeProgs(i):
        p = shows[shows['index']<=i]
        p = p.iloc[len(p)-1]
        return p['Title']
    ## Create individual records for each ad spot.  
    def expandDays(x):
        adStartDate = datetime.strptime(x['StartDate'], '%m/%d/%y')
        for i, day in enumerate(x['Weekdays']):
            if(day!='-'):
                if(day.isdigit()):
                    for j in range(0, int(day)):
                        adtimes.append({
                                    'Date': (adStartDate + timedelta(days=i)),  
                                    'Rate': x['Rate'],
                                    'Committee': committee,
                                    'Station': station,
                                    'Program': x['Program']
                                   })
                else: ## CODE FOR MTW NOTATION
                    adtimes.append({
                                    'Date': (adStartDate + timedelta(days=i)),  
                                    'Rate': x['Rate'],
                                    'Committee': committee,
                                    'Station': station,
                                    'Program': x['Program']
                                   })
    ## Filter out day times rows.                
    def getDays(row):
        #print(row)
        nextRow = 1
        dayCols = ['StartDate', 'EndDate', 'Weekdays', 'Spots/Week', 'Rate']
        for val in row:
            try:
                ind = dayCols.index(val)
                dayCols.remove(val)
            except:
                pass
        if(len(dayCols) == 0):
            if(len(headers)==0):
                headers.append(row.name)
            while (nextRow!=-1):
                dayColFound=False
                if((row.name+nextRow) != len(buys)):
                    for k in buys.loc[(row.name+nextRow), :]:
                        if k == 'Week:':
                            headers.append((row.name+nextRow))
                            nextRow=nextRow+1
                            dayColFound=True
                    if dayColFound!=True:
                        nextRow=-1
                else:
                    nextRow=-1
    def findHeaderRow(row):
        headers =  { 'Type1': ['StartDateEndDateDescription'],
                     'Type2': ['Amount', 'Start', 'InventoryCode', 'Rate', 'Spots'] }
        for htype in headers:
            for val in row:
                try:
                    ind = headers[htype].index(val)
                    headers[htype].remove(val)
                except:
                    pass
        if(len(headers['Type1']) == 0):
            return ['Description', row.name]
        else:
            if(len(headers['Type2']) == 0):
                return ['InventoryCode', row.name]
    def fixHeaders(row):
        fixCol = -1
        for k in row.keys():
            if row[k] == 'StartDateEndDateDescription':
                fixCol = k
        if fixCol != -1:
            if(fixCol == 3):
                row[(fixCol-2)] = 'Ch'
                row[(fixCol-1)] = 'Start'
                row[(fixCol)] = 'End'
                row[(fixCol+1)] = 'Description'
                row[(fixCol+2)] = 'Desc2'
            if(fixCol == 2):
                row[(fixCol)] = 'Start'
                row[(fixCol+1)] = 'End'
                row[(fixCol+2)] = 'Description'
        return row
    ## Filter out program names.
    lastIndex = 0
    for t in tables:
        progColName=False
        headerConfig = t.apply(findHeaderRow, axis=1).dropna()
        if len(headerConfig)>0:
            progColName = headerConfig.values[0][0]
        if progColName != False:
            ## Figure out inventory code lines
            progs = t.copy()
            if progColName == 'Description':
                progs = progs.apply(fixHeaders, axis=1)
                progs.columns = progs.iloc[headerConfig.values[0][1]]
            else:
                progs.columns = progs.iloc[headerConfig.values[0][1]]
            progs = progs.iloc[headerConfig.values[0][1]:]
            progs = progs.reset_index(drop=True)
            
            ## Some WideOrbit reports have two line program names, this gets the second lines.
            if progColName == 'InventoryCode':
                slines = getSecondLines(progs)
                secondlines = secondlines.append(slines)
            ## Get the program name rows
            progs = progs[(progs['Amount'] != '') & (progs['Ch'] != '')] 
            progs = progs.iloc[1:] # Remove residual header row.
            if progColName == 'Description':
                progs['Description'] = progs['Description'] + progs['Desc2']
            progs = progs.reset_index()
            ## Make index run continously accross pages.
            progs['index'] = progs['index'] + lastIndex
            lastIndex = t.iloc[len(t)-1].name+lastIndex+1
            ## Remove duplicate columns and append to the master list.
            progs = progs.loc[:,~progs.columns.duplicated()]
            progs = progs.rename(columns={progColName:'Title'})
            progs = progs[['index', 'Title']]
            shows = shows.append(progs)
        else:
            pass
            #print("False")
    ## Merge in the second lines to complete the program names.
    if progColName == 'InventoryCode':
        shows = shows.reset_index(drop=True)
        shows = shows.apply(mergeSLines, axis=1)
    ## Seperate out the days and times each ad will be on.
    lastIndex = 0
    for t in tables:
        progColName=False
        headerConfig = t.apply(findHeaderRow, axis=1).dropna()
        if len(headerConfig)>0:
            progColName = headerConfig.values[0][0]
        if progColName != False:
            headers = []
            buys = t.copy()
            buys = buys.dropna(how='all')
            buys = buys.iloc[headerConfig.values[0][1]:]
            buys = buys.reset_index(drop=True)
            #print(len(buys))
            buys.apply(getDays, axis=1)
            days = buys.iloc[headers]
            days = days.replace('', np.nan)
            days = days.dropna(axis='columns')
            days.columns = days.iloc[0]
            days = days[1:]
            days = days.reset_index()
            days['index'] = days['index'] + lastIndex
            lastIndex = t.iloc[len(t)-1].name+lastIndex+1
            ads = ads.append(days)
    ## Add in Program Names
    ads['Program'] = ads['index'].apply(mergeProgs)
    ads.apply(expandDays, axis=1)
    '''
    print('SHOWS')
    print(shows)
    print('ADS')
    print(ads[['index', 'Weekdays']])
    print(len(adtimes))
    '''
    ## Expand each ad buy listing to individual spots
    return pd.DataFrame(adtimes)

In [345]:
adtimes = adtimes.append({
                                    'Date': (adStartDate + timedelta(days=i)),  
                                    'Rate': x['Rate'],
                                    'Committee': committee,
                                    'Station': station,
                                    'Program': x['Program']
                                   }, ignore_index=True)

,Date,Rate,Committee,Station,Program


In [107]:
def parseCyclePDFs(cyc, basepath):
    fulladlist = pd.DataFrame(columns=['Date', 'Rate', 'Committee', 'Station', 'Program'])
    pdfResults = pd.DataFrame(columns=['Path', 'Result'])
    def digToNextLevel(folderpath, name):
        nonlocal pdfResults
        nonlocal fulladlist
        for item in list(os.scandir(folderpath)):
            if (item.is_file()):
                print(item.path)
                if (item.path.split('.')[1] == "pdf"):
                    result = subprocess.run(['pdftotext', item.path, '-'], 
                                            stdout=subprocess.PIPE).stdout.decode()
                    if ("INVOICE" not in result.split('\n')[0:30]): ## Check if file is a contract/order.
                        if((len(result.split('\n')[0:30])> 5)):
                            pdfTable = convertPDFToTable(item.path)
                            try: 
                                ads = processTables(pdfTable, name, station)
                                fulladlist = fulladlist.append(ads, ignore_index=True)
                                pdfResults = pdfResults.append({'Path': item.path, 
                                                                'Result': 'Success'}, ignore_index=True)
                            except:
                                 pdfResults = pdfResults.append({'Path': item.path, 
                                                    'Result': 'Scraping Error'}, ignore_index=True)
                        else:
                            pdfResults = pdfResults.append({'Path': item.path, 
                                               'Result': 'File not OCRd'}, ignore_index=True)
                    else:
                        pdfResults = pdfResults.append({'Path': item.path, 
                                               'Result': 'Not an order file.'}, ignore_index=True)
            else:
                digToNextLevel(item.path, item.name)
    for station in list(os.scandir(basepath)):
        station = station.name
        cyclepath = basepath+station+'/Political Files/'+cyc+'/'
        digToNextLevel(cyclepath, cyc)
    fulladlist['Cycle'] = cyc
    return {'ads': fulladlist, 'pdfs': pdfResults}
results = parseCyclePDFs('2018', '/home/andrew/Documents/PredictiveModeling/TV/stations/')

/home/andrew/Documents/PredictiveModeling/TV/stations/KTVK/Political Files/2018/State/KATE GALLEGO FOR PHOENIX MAYOR/Invoice.pdf
/home/andrew/Documents/PredictiveModeling/TV/stations/KTVK/Political Files/2018/State/AZ DEM PARTY JANUARY CONTRERAS/ktvw-dt.pdf
/home/andrew/Documents/PredictiveModeling/TV/stations/KTAZ/Political Files/2018/State/DAVID GARCIA/KSAZ-TV.pdf


/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


/home/andrew/Documents/PredictiveModeling/TV/stations/KPNX/Political Files/2018/State/Katie Hobbs Secretary of State 2018/ktvk.pdf


/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


/home/andrew/Documents/PredictiveModeling/TV/stations/KPNX/Political Files/2018/Federal/US Senate/Sinema 2018/ktaz.pdf


...

/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


/home/andrew/Documents/PredictiveModeling/TV/stations/KASW/Political Files/2018/Federal/US Senate/Kyrsten Sinema DSCC/kpnx.pdf


/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


/home/andrew/Documents/PredictiveModeling/TV/stations/KAET/Political Files/2018/Federal/US House/knxv-tv.pdf


/home/andrew/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [90]:
import subprocess
result = subprocess.run(['pdftotext', '/home/andrew/Documents/PredictiveModeling/TV/Invoice.pdf', '-'], 
                        stdout=subprocess.PIPE).stdout.decode()
print("ORDER" in result.split('\n')[0:10])

False


In [108]:
results['ads']

,Committee,Date,Program,Rate,Station,Cycle
0,DAVID GARCIA,2018-10-12,M-F530a-6a,$55000,KTAZ,2018
1,DAVID GARCIA,2018-10-09,M-F6a-7aNews,$92500,KTAZ,2018
2,DAVID GARCIA,2018-10-11,M-F6a-7aNews,$92500,KTAZ,2018
3,DAVID GARCIA,2018-10-15,M-F6a-7aNews,$40000,KTAZ,2018
4,DAVID GARCIA,2018-10-10,M-F6a-7aNews,$40000,KTAZ,2018
5,DAVID GARCIA,2018-10-09,M-F7a-8aNews,$92500,KTAZ,2018
6,DAVID GARCIA,2018-10-10,M-F7a-8aNews,$92500,KTAZ,2018
7,DAVID GARCIA,2018-10-11,M-F7a-8aNews,$92500,KTAZ,2018
8,DAVID GARCIA,2018-10-10,M-F7a-8aNews,$92500,KTAZ,2018
9,DAVID GARCIA,2018-10-12,M-F7a-8aNews,$92500,KTAZ,2018
